In [ ]:
import scirpy as ir

In [ ]:
ir.__version__

In [ ]:
import scanpy as sc

In [ ]:
from glob import glob
import warnings
import tarfile

import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc

In [ ]:
import muon as mu

In [ ]:
import numpy as np

In [ ]:
samples = {
    "10mix1": {"group": "10mix"},
    "10mix2": {"group": "10mix"},
    "11mix1": {"group": "11mix"},
    "11mix2": {"group": "11mix"},
    "GF1": {"group": "GF"},
    "GF2": {"group": "GF"},

}    
    
# Create a list of AnnData objects (one for each sample)
adatas_tcr = {}
adatas_gex = {}
for sample, sample_meta in samples.items():
    #adata_gex = sc.read_10x_h5(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix.h5")
    adata_tcr = ir.io.read_10x_vdj(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/vdj_t/filtered_contig_annotations.csv")
    # concatenation only works with unique gene names
    #adata_gex.var_names_make_unique()
    adatas_tcr[sample] = adata_tcr
    #adatas_gex[sample] = adata_gex

In [ ]:
samples = {
    "10mix-ICI1": {"group": "10mix", "condition":"ICI"},
    "10mix-ICI2": {"group": "10mix","condition":"ICI"},
    "11mix-ICI1": {"group": "11mix","condition":"ICI"},
    "11mix-ICI2": {"group": "11mix","condition":"ICI"},
    "GF-ICI1": {"group": "GF","condition":"No ICI"},
    "GF-ICI2": {"group": "GF","condition":"No ICI"},
    "GF-ICI1-plus": {"group": "GF-plus","condition":"ICI"},
    "GF-ICI2-plus": {"group": "GF-plus","condition":"ICI"},


}    
    
# Create a list of AnnData objects (one for each sample)
adatas_tcr_2021 = {}
#adatas_gex_2021 = {}
for sample, sample_meta in samples.items():
    #adata_gex_2021 = sc.read_10x_h5(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix.h5")
    adata_tcr_2021 = ir.io.read_10x_vdj(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/vdj_t/filtered_contig_annotations.csv")
    # concatenation only works with unique gene names
    #adata_gex_2021.var_names_make_unique()
    adatas_tcr_2021[sample] = adata_tcr_2021
    #adatas_gex_2021[sample] = adata_gex_2021

In [ ]:
samples = {
    "10mix-ICI1": {"group": "10mix", "condition":"ICI"},
    "10mix-ICI2": {"group": "10mix","condition":"ICI"},
    "11mix-ICI1": {"group": "11mix","condition":"ICI"},
    "11mix-ICI2": {"group": "11mix","condition":"ICI"},
    "GF-ICI1": {"group": "GF","condition":"No ICI"},
    "GF-ICI2": {"group": "GF","condition":"No ICI"},
    "GF-ICI1-plus": {"group": "GF-plus","condition":"ICI"},
    "GF-ICI2-plus": {"group": "GF-plus","condition":"ICI"},
    "10mix1": {"group": "10mix", "condition":"naive"},
    "10mix2": {"group": "10mix", "condition":"naive"},
    "11mix1": {"group": "11mix", "condition":"naive"},
    "11mix2": {"group": "11mix", "condition":"naive"},
    "GF1": {"group": "GF", "condition":"naive"},
    "GF2": {"group": "GF", "condition":"naive"},
}    

### TCR 2019 NORMAL

In [ ]:
adata_tcr

In [ ]:
# Merge anndata objects
adata_tcr = anndata.concat(adatas_tcr, index_unique="_")

In [ ]:
adata_tcr.obs

### TCR 2021 TUMOR

In [ ]:
adata_tcr_2021

In [ ]:
# Merge anndata objects
adata_tcr_2021 = anndata.concat(adatas_tcr_2021, index_unique="_")

In [ ]:
adata_tcr_2021.obs

### Load GEX

In [ ]:
adata = sc.read_h5ad("mapped_til_colon.h5ad")

In [ ]:
adata.var

In [ ]:
adata.obs.origin =  adata.obs.origin.replace("colon","normal")

In [ ]:
adata.obs.origin =  adata.obs.origin.replace("til","tumor")

In [ ]:
adata_normal = adata[adata.obs["origin"]=="normal"]

In [ ]:
adata_tumor = adata[adata.obs["origin"]=="tumor"]

### Transform TCR 2019 NORMAL

In [ ]:
adata_tcr.obs["sample_id"] = adata_tcr.obs_names.to_series().str.split("_", expand=True)[1]

In [ ]:
adata_tcr.obs["group"] = adata_tcr.obs["sample_id"].map(lambda x: samples[x]["group"])
adata_tcr.obs["condition"] = adata_tcr.obs["sample_id"].map(lambda x: samples[x]["condition"])

In [ ]:
adata_tcr.obs

### Transform TCR 2021 TUMOR

In [ ]:
adata_tcr_2021.obs["sample_id"] = adata_tcr_2021.obs_names.to_series().str.split("_", expand=True)[1]

In [ ]:
adata_tcr_2021.obs["group"] = adata_tcr_2021.obs["sample_id"].map(lambda x: samples[x]["group"])
adata_tcr_2021.obs["condition"] = adata_tcr_2021.obs["sample_id"].map(lambda x: samples[x]["condition"])

In [ ]:
adata_tcr_2021.obs

### Merge TCR TUMOR & NORMAL

In [ ]:
adata_tcr_merge = anndata.concat([adata_tcr,adata_tcr_2021], index_unique="_")

In [ ]:
adata_tcr_merge.obs

### Merge GEX TUMOR & NORMAL

In [ ]:
adata_gex_merge = anndata.concat([adata_normal,adata_tumor], index_unique="_")

In [ ]:
adata_gex_merge.obs

### Create MUDATA

In [ ]:
mdata = mu.MuData({"gex": adata_gex_merge, "airr": adata_tcr_merge})

### Transfor obs names for gex and tcr matching

In [ ]:
mdata.obs.index = mdata.obs.index.str.replace('-', '_')

In [ ]:
# Replace "-" with "_" in the 'obs_names' column
mdata["airr"].obs_names = mdata["airr"].obs_names.str.replace('-', '_')

In [ ]:
# Replace "-" with "_" in the 'obs_names' column
mdata["gex"].obs_names = mdata["gex"].obs_names.str.replace('-', '_')

In [ ]:
len(set(mdata["gex"].obs_names) & set(mdata["airr"].obs_names))

In [ ]:
mdata

In [ ]:
sc.pl.umap(mdata["gex"], color = "leiden")

### Create MDATA GEX & TCR 2019 NORMAL

In [ ]:
mdata_normal = mu.MuData({"gex": adata_normal, "airr": adata_tcr})

In [ ]:
mdata_normal

In [ ]:
sc.pl.umap(mdata_normal["gex"], color = "leiden")

### Create MDATA GEX & TCR 2021 TUMOR

In [ ]:
mdata_tumor= mu.MuData({"gex": adata_tumor, "airr": adata_tcr_2021})

In [ ]:
mdata_tumor

In [ ]:
sc.pl.umap(mdata_tumor["gex"], color = "leiden")

### Transform mdata

In [ ]:
sc.pp.filter_genes(mdata["gex"], min_cells=500)
sc.pp.filter_cells(mdata["gex"], min_genes=500)

In [ ]:
sc.pp.log1p(mdata["gex"])
sc.pp.pca(mdata["gex"], svd_solver="arpack")
sc.pp.neighbors(mdata["gex"])
sc.tl.umap(mdata["gex"])

In [ ]:
ir.pp.index_chains(mdata)
ir.tl.chain_qc(mdata)

In [ ]:
# Define the conditions
normal_samples = ["10mix1", "10mix2", "11mix1", "11mix2", "GF1", "GF2"]

# Use np.where() to create the 'origin' column based on conditions
mdata["airr"].obs["origin"] = mdata["airr"].obs["sample_id"].apply(
    lambda x: "colon" if x in normal_samples else "tumor"
)

In [ ]:
mdata

In [ ]:
mdata["gex"].obs.index

In [ ]:
mdata["airr"].obs.index

In [ ]:
mdata.obs.index = mdata.obs.index.str.replace('-', '_', regex=False)


In [ ]:
mdata["airr"].obs

In [ ]:
mdata.obs

In [ ]:
mdata.update()

In [ ]:
#mdata = mu.MuData({"gex": adata_gex_merge, "airr": adata_tcr_merge})

In [ ]:
mdata["gex"].var

In [ ]:
mdata["gex"].obs.cell_type = mdata["gex"].obs.cell_type.replace("nan","CD8_NaiveLike")

In [ ]:
mdata.update()

In [ ]:
fig, (ax0, ax1,ax2,ax3) = plt.subplots(1, 4, figsize=(20, 4), gridspec_kw={"wspace": 0.5})
mu.pl.embedding(mdata, basis="gex:umap", color=["ENSMUSG00000055170"], ax=ax0, show=False, frameon=False, cmap= "Reds")
mu.pl.embedding(mdata, basis="gex:umap", color=["airr:origin"], ax=ax1, show=False, frameon=False)
mu.pl.embedding(mdata, basis="gex:umap", color=["gex:cell_type"], ax=ax2, show=False, frameon=False)
mu.pl.embedding(mdata, basis="gex:umap", color=["airr:receptor_subtype"], ax=ax3, frameon=False)

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2,figsize=(10, 5) ,gridspec_kw={"wspace": 0.5})
mu.pl.embedding(mdata, basis="gex:umap", color=["ENSMUSG00000055170"], ax=ax0, show=False, frameon=False, cmap= "Reds")
mu.pl.embedding(mdata, basis="gex:umap", color=["gex:cell_type"], ax=ax1, show=False, frameon=False)


In [ ]:
ir.tl.chain_qc(mdata)

_ = ir.pl.group_abundance(
   mdata, groupby="airr:receptor_type", target_col="gex:sample_id"
)


In [ ]:
ir.tl.chain_qc(mdata)

_ = ir.pl.group_abundance(
   mdata, groupby="airr:receptor_type", target_col="gex:cell_type"
)


In [ ]:
ir.tl.chain_qc(mdata)

_ = ir.pl.group_abundance(
   mdata, groupby="airr:receptor_type", target_col="gex:condition"
)


In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:origin")

In [ ]:
print(
    "Fraction of cells with more than one pair of TCRs: {:.2f}".format(
        np.sum(mdata.obs["airr:chain_pairing"].isin(["extra VJ", "extra VDJ", "two full chains", "multichain"]))
        / mdata["airr"].n_obs
    )
)

In [ ]:
mu.pl.embedding(mdata, basis="gex:umap", color="airr:chain_pairing", groups="multichain")

In [ ]:
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: x != "multichain")

In [ ]:
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: ~np.isin(x, ["orphan VDJ", "orphan VJ"]))

In [ ]:
mdata

In [ ]:
ax = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:origin")

In [ ]:
# using default parameters, `ir_dist` will compute nucleotide sequence identity
ir.pp.ir_dist(mdata)


In [ ]:
mdata["airr"].obs["chain_pairing"]=mdata["airr"].obs["chain_pairing"].astype("string")

In [ ]:
mdata["airr"].obs.info()

In [ ]:
ir.tl.define_clonotypes(mdata, receptor_arms="all", dual_ir="primary_only")

In [ ]:
ir.pp.ir_dist(
    mdata,
    metric="alignment",
    sequence="aa",
    cutoff=15,
)

In [ ]:
ir.tl.define_clonotype_clusters(mdata, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any")

In [ ]:
ir.tl.clonotype_network(mdata, min_cells=2, sequence="aa", metric="alignment")

In [ ]:
_ = ir.pl.clonotype_network(
    mdata,
    color="gex:sample_id",
    label_fontsize=9,
    panel_size=(7, 7),
    base_size=2
)


In [ ]:
_ = ir.pl.clonal_expansion(
    mdata,
    target_col="clone_id",
    groupby="gex:cell_type"
)


In [ ]:
_ = ir.pl.alpha_diversity(
    mdata,
    metric="normalized_shannon_entropy",
    groupby="gex:cell_type"
)


In [ ]:
ir.tl.clonotype_modularity(mdata, target_col="airr:cc_aa_alignment")

In [ ]:
_ = ir.pl.clonotype_modularity(mdata, base_size=20)

In [ ]:
clonotypes_top_modularity = list(
	mdata.obs.set_index("airr:cc_aa_alignment")["airr:clonotype_modularity"]
	.sort_values(ascending=False)
	.index.unique()
	.values[:1]
)

In [ ]:
mu.pl.embedding(mdata,basis="gex:tsne", color=["gex:cell_type"])
mu.pl.embedding(
    mdata,
    basis="gex:tsne",
    color="airr:cc_aa_alignment",
    groups=clonotypes_top_modularity
)
